# Classification of Ghouls , Goblins and Ghosts

## Data Preprocessing

### Importing Libraries

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
dataset_training = pd.read_csv("/kaggle/input/ghouls-goblins-and-ghosts-boo/train.csv.zip")
dataset_test = pd.read_csv("/kaggle/input/ghouls-goblins-and-ghosts-boo/test.csv.zip")

In [ ]:
import seaborn as sns

In [ ]:
sns.pairplot(dataset_training, hue='type')

### Splitting the dataset into dependent and predicted variable

In [ ]:
X = dataset_training.drop(['id', 'type'], axis=1)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [-1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [ ]:
y = dataset_training['type'].values

In [ ]:
ohe = OneHotEncoder(sparse=False)
y = ohe.fit_transform(y.reshape(-1, 1))

In [ ]:
X

In [ ]:
y

### Splitting the data into training and testing data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part-2 Building a ANN for detecting Ghouls, Goblins and Ghosts

In [ ]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=30, activation='relu'))
ann.add(tf.keras.layers.Dense(units=30, activation='relu'))
ann.add(tf.keras.layers.Dense(units=30, activation='relu'))
ann.add(tf.keras.layers.Dense(units=3, activation='softmax'))
ann.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
ann.fit(X_train, y_train, batch_size=32, epochs=40, validation_data=(X_test, y_test))

## Part -3 predicting the result of the test set

In [ ]:
X_submission = dataset_test.drop('id', axis=1)
X_submission = np.array(ct.transform(X_submission))
X_submission = sc.transform(X_submission)
y_submission = ann.predict(X_submission)
y_pred = y_submission > 0.5
y_submission = ohe.inverse_transform(y_pred)
y_submission = y_submission.reshape(len(y_submission))
dataset_submission = pd.DataFrame({'id': dataset_test['id'].values, 'type': y_submission})
dataset_submission = dataset_submission.set_index('id')
dataset_submission.to_csv('Submission.csv')